### 1. 🚀 서버 모드 (Docker로 실행된 서버에 연결)

In [7]:
# QdrantClient 초기화
from qdrant_client import QdrantClient

# gRPC로 연결 (성능이 더 좋음)
client = QdrantClient("localhost", port=6334, prefer_grpc=True)
print("gRPC를 통해 로컬 Qdrant 서버에 연결되었습니다.")

gRPC를 통해 로컬 Qdrant 서버에 연결되었습니다.


### 2. 🗃️ 컬렉션 생성 및 관리 (세부 파라미터 설명 포함)

#### 기본 컬렉션 생성

컬렉션을 생성할 때는 다음과 같은 필수 파라미터를 지정해야 합니다:

- 벡터 크기(size): 벡터의 차원
- 거리 메트릭(distance): 벡터 간 유사도 계산 방법

In [8]:
from qdrant_client.http import models

# 기본 컬렉션 생성 (512차원, 코사인 유사도)
collection_name = "my_first_collection"

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=512,  # 벡터 차원
        distance=models.Distance.COSINE  # 거리 메트릭
    )
)

# 생성된 컬렉션 정보 확인
collection_info = client.get_collection(collection_name)
print(f"컬렉션 '{collection_name}' 생성 완료!")
print(f"상태: {collection_info.status}")
print(f"벡터 크기: {collection_info.config.params.vectors.size}")
print(f"거리 메트릭: {collection_info.config.params.vectors.distance}")

컬렉션 'my_first_collection' 생성 완료!
상태: green
벡터 크기: 512
거리 메트릭: Cosine


#### 고급 컬렉션 설정

Qdrant는 다양한 고급 설정을 지원합니다. 여기서는 주요 파라미터들을 살펴보겠습니다:

- HNSW 인덱스 설정: 검색 속도와 정확도 조정
- 최적화 설정: 인덱싱 임계값, 세그먼트 관리 등
- 샤딩 설정: 분산 배포를 위한 샤드 수
- 온디스크 설정: 메모리 사용량 최적화

In [9]:
# 고급 설정이 적용된 컬렉션 생성
advanced_collection = "advanced_collection"

client.create_collection(
    collection_name=advanced_collection,
    vectors_config=models.VectorParams(
        size=384,
        distance=models.Distance.COSINE
    ),
    # HNSW 인덱스 설정
    hnsw_config=models.HnswConfigDiff(
        m=16,                     # 그래프의 각 노드에서 연결되는 이웃 수 (기본값: 16)
        ef_construct=100,         # 인덱스 구축 중 고려할 이웃 수 (기본값: 100)
        full_scan_threshold=10000, # 이 임계값 이하의 벡터 수에서는 전체 스캔 사용
        max_indexing_threads=0    # 0은 모든 사용 가능한 CPU 사용
    ),
    # 최적화 설정
    optimizers_config=models.OptimizersConfigDiff(
        deleted_threshold=0.2,    # 세그먼트 최적화 임계값 (삭제된 벡터 비율)
        vacuum_min_vector_number=1000,  # 최적화 최소 벡터 수
        default_segment_number=0, # 기본 세그먼트 수 (0은 자동)
        indexing_threshold=20000, # 인덱싱 임계값 (바이트)
        flush_interval_sec=5,     # 플러시 간격 (초)
        max_optimization_threads=1 # 최적화에 사용할 최대 스레드 수
    ),
    # 샤딩 설정 (분산 환경에서 유용)
    shard_number=1,
    # 온디스크 페이로드 설정 (메모리 사용량 최적화)
    on_disk_payload=False
)

print(f"고급 설정이 적용된 컬렉션 '{advanced_collection}' 생성 완료!")

고급 설정이 적용된 컬렉션 'advanced_collection' 생성 완료!


#### 다중 벡터 컬렉션

Qdrant는 하나의 포인트에 여러 개의 벡터를 저장할 수 있습니다. 이는 하나의 문서에 대해 여러 임베딩을 사용하는 경우에 유용합니다.

In [10]:
# 다중 벡터 컬렉션 생성
multi_vector_collection = "multi_vector_collection"

client.create_collection(
    collection_name=multi_vector_collection,
    vectors_config={
        "text": models.VectorParams(
            size=384,
            distance=models.Distance.COSINE
        ),
        "image": models.VectorParams(
            size=512,
            distance=models.Distance.EUCLID
        )
    }
)

print(f"다중 벡터 컬렉션 '{multi_vector_collection}' 생성 완료!")

다중 벡터 컬렉션 'multi_vector_collection' 생성 완료!


#### 희소 벡터 컬렉션

Qdrant는 희소 벡터(Sparse Vector)도 지원합니다. 희소 벡터는 대부분의 요소가 0인 벡터로, 텍스트 검색에서 특히 유용합니다.

In [11]:
# 희소 벡터가 포함된 컬렉션 생성
sparse_collection = "sparse_collection"

client.create_collection(
    collection_name=sparse_collection,
    vectors_config={
        "dense": models.VectorParams(
            size=384,
            distance=models.Distance.COSINE
        )
    },
    sparse_vectors_config={
        "sparse": models.SparseVectorParams()
    }
)

print(f"희소 벡터 컬렉션 '{sparse_collection}' 생성 완료!")

희소 벡터 컬렉션 'sparse_collection' 생성 완료!


### 3. 컬렉션 관리

기존 컬렉션을 조회, 업데이트, 삭제하는 방법을 알아봅시다.

In [12]:
# 모든 컬렉션 목록 조회
collections = client.get_collections()
print("컬렉션 목록:")
for collection in collections.collections:
    print(f" - {collection.name}")

# 컬렉션 매개변수 업데이트 (예: 인덱싱 임계값 변경)
client.update_collection(
    collection_name=collection_name,
    optimizers_config=models.OptimizersConfigDiff(
        indexing_threshold=50000  # 50KB로 임계값 변경
    )
)
print(f"컬렉션 '{collection_name}' 업데이트 완료!")

# 컬렉션 삭제
# client.delete_collection(collection_name="컬렉션_이름")
# print("컬렉션 삭제 완료!")

컬렉션 목록:
 - sparse_collection
 - advanced_collection
 - multi_vector_collection
 - my_first_collection
컬렉션 'my_first_collection' 업데이트 완료!


### 4. 벡터 추가 및 관리

Qdrant에서는 벡터와 관련 메타데이터를 포인트(Point)로 저장합니다. 각 포인트는 벡터와 페이로드(메타데이터)로 구성됩니다.

#### 단일 포인트 추가

In [13]:
import numpy as np
import random

# 랜덤 벡터 생성
vector_size = 512
random_vector = np.random.rand(vector_size).tolist()

# 단일 포인트 추가
client.upsert(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=1,  # 포인트 ID (정수 또는 UUID 문자열 가능)
            vector=random_vector,  # 벡터 데이터
            payload={  # 메타데이터 (페이로드)
                "title": "첫 번째 문서",
                "text": "이것은 Qdrant에 추가한 첫 번째 문서입니다.",
                "category": "tutorial",
                "rating": 5,
                "tags": ["qdrant", "vector", "database"]
            }
        )
    ]
)

print("포인트 추가 완료!")

포인트 추가 완료!


#### 여러 포인트 일괄 추가

In [14]:
# 여러 포인트 생성
batch_size = 5
points = []

for i in range(2, 2 + batch_size):
    # 랜덤 벡터 생성
    vector = np.random.rand(vector_size).tolist()
    
    # 카테고리 랜덤 선택
    categories = ["tutorial", "guide", "article", "news", "blog"]
    category = random.choice(categories)
    
    # 평점 랜덤 생성 (1~5)
    rating = random.randint(1, 5)
    
    # 태그 랜덤 선택
    all_tags = ["qdrant", "vector", "database", "python", "tutorial", "ai", "ml", "search"]
    tags = random.sample(all_tags, random.randint(1, 4))
    
    # 포인트 생성
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload={
            "title": f"문서 #{i}",
            "text": f"이것은 Qdrant 튜토리얼 샘플 문서 #{i}입니다.",
            "category": category,
            "rating": rating,
            "tags": tags
        }
    )
    
    points.append(point)

# 여러 포인트 일괄 추가
client.upsert(
    collection_name=collection_name,
    points=points
)

print(f"{batch_size}개의 포인트를 일괄 추가했습니다.")

5개의 포인트를 일괄 추가했습니다.


#### 다중 벡터 포인트 추가

다중 벡터 컬렉션에는 하나의 포인트에 여러 개의 벡터를 포함시킬 수 있습니다.

In [15]:
# 다중 벡터 포인트 추가
text_vector = np.random.rand(384).tolist()  # 텍스트 벡터
image_vector = np.random.rand(512).tolist()  # 이미지 벡터

client.upsert(
    collection_name=multi_vector_collection,
    points=[
        models.PointStruct(
            id=1,
            vector={
                "text": text_vector,
                "image": image_vector
            },
            payload={
                "title": "다중 벡터 예제",
                "description": "텍스트와 이미지 벡터가 모두 포함된 예제입니다.",
                "url": "https://example.com/document",
                "timestamp": "2023-01-01T12:00:00Z"
            }
        )
    ]
)

print("다중 벡터 포인트 추가 완료!")

다중 벡터 포인트 추가 완료!


#### 포인트 조회 및 관리

In [19]:
# 특정 ID의 포인트 조회
retrieved_point = client.retrieve(
    collection_name=collection_name,
    ids=[1],
    with_payload=True,  # 페이로드 포함
    with_vectors=True   # 벡터 포함
)

print("조회된 포인트:")
print(f"ID: {retrieved_point[0].id}")
print(f"페이로드: {retrieved_point[0].payload}")
print(f"벡터 차원: {len(retrieved_point[0].vector)}")

# 포인트 수 확인
count = client.count(
    collection_name=collection_name,
    exact=True  # 정확한 수 계산
)
print(f"컬렉션 '{collection_name}'의 포인트 수: {count.count}")

# 포인트 삭제
# client.delete(
#     collection_name=collection_name,
#     points_selector=models.PointIdsList(
#         points=[1, 2, 3]  # 삭제할 포인트 ID 목록
#     )
# )
# print("포인트 삭제 완료!")

조회된 포인트:
ID: 1
페이로드: {'rating': 5, 'title': '첫 번째 문서', 'tags': ['qdrant', 'vector', 'database'], 'text': '이것은 Qdrant에 추가한 첫 번째 문서입니다.', 'category': 'tutorial'}
벡터 차원: 512
컬렉션 'my_first_collection'의 포인트 수: 6


#### 검색 기능 활용 (유사도 검색, 필터링 등)

Qdrant의 핵심 기능은 벡터 유사도 검색입니다. 이 섹션에서는 검색 기능과 다양한 옵션을 살펴봅니다.

#### 기본 유사도 검색

In [20]:
# 검색용 쿼리 벡터 생성
query_vector = np.random.rand(vector_size).tolist()

# 기본 유사도 검색
search_results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3  # 상위 3개 결과만 반환
)

print("검색 결과:")
for result in search_results:
    print(f"ID: {result.id}, 유사도 점수: {result.score}")
    
# 페이로드와 함께 결과 조회
search_results_with_payload = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3,
    with_payload=True  # 페이로드 포함
)

print("\n페이로드를 포함한 검색 결과:")
for result in search_results_with_payload:
    print(f"ID: {result.id}, 유사도 점수: {result.score}")
    print(f"제목: {result.payload.get('title')}")
    print(f"카테고리: {result.payload.get('category')}")
    print(f"태그: {result.payload.get('tags')}")
    print("-" * 40)

검색 결과:
ID: 1, 유사도 점수: 0.7526421546936035
ID: 6, 유사도 점수: 0.7412857413291931
ID: 4, 유사도 점수: 0.740224301815033

페이로드를 포함한 검색 결과:
ID: 1, 유사도 점수: 0.7526421546936035
제목: 첫 번째 문서
카테고리: tutorial
태그: ['qdrant', 'vector', 'database']
----------------------------------------
ID: 6, 유사도 점수: 0.7412857413291931
제목: 문서 #6
카테고리: article
태그: ['qdrant', 'search', 'ai', 'python']
----------------------------------------
ID: 4, 유사도 점수: 0.740224301815033
제목: 문서 #4
카테고리: article
태그: ['qdrant', 'vector', 'ai', 'python']
----------------------------------------


#### 필터링을 적용한 검색

Qdrant는 강력한 필터링 기능을 제공합니다. 필터를 사용하여 특정 조건을 만족하는 결과만 검색할 수 있습니다.

In [21]:
# 카테고리가 'tutorial'인 포인트만 검색
filter_category = models.Filter(
    must=[
        models.FieldCondition(
            key="category",
            match=models.MatchValue(value="tutorial")
        )
    ]
)

search_with_filter = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    query_filter=filter_category,  # filter → query_filter로 변경
    limit=3,
    with_payload=True
)

print("카테고리 'tutorial'로 필터링된 검색 결과:")
for result in search_with_filter:
    print(f"ID: {result.id}, 유사도 점수: {result.score}")
    print(f"제목: {result.payload.get('title')}")
    print(f"카테고리: {result.payload.get('category')}")
    print("-" * 40)

# 복합 필터 예제: 평점이 4 이상이고 'qdrant' 태그가 있는 포인트 검색
complex_filter = models.Filter(
    must=[
        # 평점이 4 이상
        models.FieldCondition(
            key="rating",
            range=models.Range(
                gte=4  # Greater than or equal to 4
            )
        ),
        # 'qdrant' 태그 포함
        models.FieldCondition(
            key="tags",
            match=models.MatchValue(value="qdrant")
        )
    ]
)

search_with_complex_filter = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    query_filter=complex_filter,  # filter → query_filter로 변경
    limit=3,
    with_payload=True
)

print("\n복합 필터링된 검색 결과 (평점 4 이상 & 'qdrant' 태그):")
for result in search_with_complex_filter:
    print(f"ID: {result.id}, 유사도 점수: {result.score}")
    print(f"제목: {result.payload.get('title')}")
    print(f"평점: {result.payload.get('rating')}")
    print(f"태그: {result.payload.get('tags')}")
    print("-" * 40)

카테고리 'tutorial'로 필터링된 검색 결과:
ID: 1, 유사도 점수: 0.7526421546936035
제목: 첫 번째 문서
카테고리: tutorial
----------------------------------------
ID: 5, 유사도 점수: 0.7335762977600098
제목: 문서 #5
카테고리: tutorial
----------------------------------------

복합 필터링된 검색 결과 (평점 4 이상 & 'qdrant' 태그):
ID: 1, 유사도 점수: 0.7526421546936035
제목: 첫 번째 문서
평점: 5
태그: ['qdrant', 'vector', 'database']
----------------------------------------
ID: 4, 유사도 점수: 0.740224301815033
제목: 문서 #4
평점: 5
태그: ['qdrant', 'vector', 'ai', 'python']
----------------------------------------


#### 고급 검색 옵션

Qdrant는 다양한 고급 검색 옵션을 제공합니다. 여기서는 몇 가지 중요한 옵션을 살펴보겠습니다.



In [22]:
# 정확도와 속도 조정 (hnsw_ef 파라미터)
search_precise = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3,
    with_payload=True,
    search_params=models.SearchParams(hnsw_ef=128)  # 높을수록 더 정확하지만 느림 (기본값보다 높게 설정)
)

print("정확도를 높인 검색 결과 (hnsw_ef=128):")
for result in search_precise:
    print(f"ID: {result.id}, 유사도 점수: {result.score}")

# 정확한 검색 결과 (근사치 알고리즘 사용 안 함)
search_exact = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3,
    with_payload=True,
    search_params=models.SearchParams(exact=True)  # 근사치 알고리즘을 사용하지 않고 정확한 결과 계산
)

print("\n정확한 검색 결과 (exact=True):")
for result in search_exact:
    print(f"ID: {result.id}, 유사도 점수: {result.score}")

# 점수 임계값 설정
search_threshold = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=10,  # 최대 10개 결과 요청
    with_payload=True,
    score_threshold=0.7  # 유사도 점수가 0.7 이상인 결과만 반환
)

print("\n점수 임계값이 적용된 검색 결과 (score_threshold=0.7):")
for result in search_threshold:
    print(f"ID: {result.id}, 유사도 점수: {result.score}")

정확도를 높인 검색 결과 (hnsw_ef=128):
ID: 1, 유사도 점수: 0.7526421546936035
ID: 6, 유사도 점수: 0.7412857413291931
ID: 4, 유사도 점수: 0.740224301815033

정확한 검색 결과 (exact=True):
ID: 1, 유사도 점수: 0.7526421546936035
ID: 6, 유사도 점수: 0.7412857413291931
ID: 4, 유사도 점수: 0.740224301815033

점수 임계값이 적용된 검색 결과 (score_threshold=0.7):
ID: 1, 유사도 점수: 0.7526421546936035
ID: 6, 유사도 점수: 0.7412857413291931
ID: 4, 유사도 점수: 0.740224301815033
ID: 5, 유사도 점수: 0.7335762977600098
ID: 3, 유사도 점수: 0.7314790487289429
ID: 2, 유사도 점수: 0.7295136451721191


#### 포인트 ID로 검색

이미 저장된 포인트의 ID를 사용하여 유사한 포인트를 검색할 수 있습니다.



In [23]:
# 포인트 ID로 검색 (포인트 ID 1과 유사한 포인트 검색)
search_by_id = client.recommend(
    collection_name=collection_name,
    positive=[1],  # 기존 포인트 ID (추천 기준 포인트)
    limit=3,
    with_payload=True
)

print("포인트 ID 1과 유사한 검색 결과:")
for result in search_by_id:
    print(f"ID: {result.id}, 유사도 점수: {result.score}")
    print(f"제목: {result.payload.get('title')}")
    print("-" * 40)

포인트 ID 1과 유사한 검색 결과:
ID: 3, 유사도 점수: 0.771040141582489
제목: 문서 #3
----------------------------------------
ID: 6, 유사도 점수: 0.7631308436393738
제목: 문서 #6
----------------------------------------
ID: 2, 유사도 점수: 0.7566003799438477
제목: 문서 #2
----------------------------------------


#### 다중 벡터 검색

다중 벡터 컬렉션에서 특정 벡터 이름을 지정하여 검색할 수 있습니다.

In [24]:
# 다중 벡터 컬렉션에서 텍스트 벡터로 검색
text_query = np.random.rand(384).tolist()

text_vector_search = client.search(
    collection_name=multi_vector_collection,
    query_vector=("text", text_query),  # 튜플 형태로 벡터 이름과 벡터 전달
    limit=3,
    with_payload=True
)

print("텍스트 벡터를 사용한 검색 결과:")
for result in text_vector_search:
    print(f"ID: {result.id}, 유사도 점수: {result.score}")
    print(f"제목: {result.payload.get('title')}")
    print("-" * 40)

텍스트 벡터를 사용한 검색 결과:
ID: 1, 유사도 점수: 0.7475517392158508
제목: 다중 벡터 예제
----------------------------------------


### LangChain과 Qdrant 연동 방법

이 섹션에서는 LangChain과 Qdrant를 연동하는 방법을 알아봅니다. LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 프레임워크로, Qdrant를 벡터 저장소로 사용할 수 있습니다.

#### 환경 설정

먼저 필요한 패키지가 설치되어 있는지 확인합니다.

In [26]:
try:
    from langchain_qdrant import QdrantVectorStore
    from langchain_ollama import OllamaEmbeddings
    from langchain_community.document_loaders import TextLoader
    from langchain.text_splitter import CharacterTextSplitter
    print("LangChain과 관련 패키지가 정상적으로 임포트되었습니다.")
except ImportError as e:
    print(f"패키지 설치가 필요합니다: {e}")
    print("다음 명령어로 설치하세요: pip install langchain-qdrant langchain-ollama langchain-community")

LangChain과 관련 패키지가 정상적으로 임포트되었습니다.


#### QdrantVectorStore 초기화

In [29]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import Distance, VectorParams

# OpenAI 임베딩 모델 초기화
embeddings = OllamaEmbeddings(model="bge-m3")

# 새로운 Qdrant 컬렉션 이름 설정
langchain_collection = "langchain_collection"

# Qdrant 클라이언트 생성
client = QdrantClient(":memory:")

# 컬렉션 생성
client.create_collection(
    collection_name=langchain_collection,
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

# QdrantVectorStore 초기화 (메모리 모드)
vector_store = QdrantVectorStore(
    client=client,
    collection_name=langchain_collection,
    embedding=embeddings
)

print(f"QdrantVectorStore가 {langchain_collection} 컬렉션으로 초기화되었습니다.")

QdrantVectorStore가 langchain_collection 컬렉션으로 초기화되었습니다.


#### 문서 추가

이제 문서를 벡터 저장소에 추가해 보겠습니다.



In [30]:
from langchain_core.documents import Document
from uuid import uuid4

# 예제 문서 생성
documents = [
    Document(
        page_content="Qdrant는 벡터 유사도 검색을 위한 오픈소스 벡터 데이터베이스입니다. 러스트로 작성되었으며 고성능을 자랑합니다.",
        metadata={"source": "documentation", "topic": "vector_database", "author": "qdrant_team"}
    ),
    Document(
        page_content="LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션 개발을 위한 프레임워크입니다.",
        metadata={"source": "documentation", "topic": "framework", "author": "langchain_team"}
    ),
    Document(
        page_content="RAG(Retrieval Augmented Generation)는 대규모 언어 모델의 성능을 향상시키기 위해 외부 지식을 가져오는 방법입니다.",
        metadata={"source": "research_paper", "topic": "llm", "author": "ai_researcher"}
    ),
    Document(
        page_content="벡터 데이터베이스는 임베딩을 저장하고 효율적으로 검색하기 위한 특수 데이터베이스입니다.",
        metadata={"source": "blog", "topic": "vector_database", "author": "tech_blogger"}
    ),
    Document(
        page_content="HNSW(Hierarchical Navigable Small World)는 고차원 벡터의 근사 최근접 이웃 검색을 위한 알고리즘입니다.",
        metadata={"source": "research_paper", "topic": "algorithm", "author": "algorithm_researcher"}
    )
]

# 문서 ID 생성
ids = [str(uuid4()) for _ in range(len(documents))]

# 문서를 벡터 저장소에 추가
vector_store.add_documents(documents=documents, ids=ids)

print(f"{len(documents)}개의 문서가 벡터 저장소에 추가되었습니다.")

5개의 문서가 벡터 저장소에 추가되었습니다.


#### 유사도 검색

이제 LangChain의 인터페이스를 통해 유사도 검색을 수행해 보겠습니다.



In [31]:
# 기본 유사도 검색
query = "벡터 데이터베이스란 무엇인가요?"
docs = vector_store.similarity_search(query, k=2)

print(f"검색 쿼리: '{query}'")
print("검색 결과:")
for i, doc in enumerate(docs):
    print(f"{i+1}. {doc.page_content}")
    print(f"   메타데이터: {doc.metadata}")
    print("-" * 50)

# 유사도 점수가 포함된 검색
docs_and_scores = vector_store.similarity_search_with_score(query, k=2)

print("\n점수가 포함된 검색 결과:")
for i, (doc, score) in enumerate(docs_and_scores):
    print(f"{i+1}. 점수: {score}")
    print(f"   내용: {doc.page_content}")
    print(f"   메타데이터: {doc.metadata}")
    print("-" * 50)

검색 쿼리: '벡터 데이터베이스란 무엇인가요?'
검색 결과:
1. 벡터 데이터베이스는 임베딩을 저장하고 효율적으로 검색하기 위한 특수 데이터베이스입니다.
   메타데이터: {'source': 'blog', 'topic': 'vector_database', 'author': 'tech_blogger', '_id': 'f88deefa-f6da-4842-9284-927ccbedd63a', '_collection_name': 'langchain_collection'}
--------------------------------------------------
2. Qdrant는 벡터 유사도 검색을 위한 오픈소스 벡터 데이터베이스입니다. 러스트로 작성되었으며 고성능을 자랑합니다.
   메타데이터: {'source': 'documentation', 'topic': 'vector_database', 'author': 'qdrant_team', '_id': '7dff5e89-fa27-41c4-b5f6-e29a4aaff2ad', '_collection_name': 'langchain_collection'}
--------------------------------------------------

점수가 포함된 검색 결과:
1. 점수: 0.7468298660494885
   내용: 벡터 데이터베이스는 임베딩을 저장하고 효율적으로 검색하기 위한 특수 데이터베이스입니다.
   메타데이터: {'source': 'blog', 'topic': 'vector_database', 'author': 'tech_blogger', '_id': 'f88deefa-f6da-4842-9284-927ccbedd63a', '_collection_name': 'langchain_collection'}
--------------------------------------------------
2. 점수: 0.5780042436225572
   내용: Qdrant는 벡터 유사도 검색을 위한 오픈소스 벡터 데이

#### 메타데이터 필터링

LangChain을 통해 메타데이터 필터링을 사용할 수 있습니다.



In [38]:
# 메타데이터 필터를 적용한 검색
filtered_docs = vector_store.similarity_search(
    query="벡터 데이터베이스란 무엇인가요?",
    k=2,
    filter=models.Filter(
        should=[
            models.FieldCondition(
                key="topic",
                match=models.MatchValue(
                    value="vector_database"
                ),
            ),
        ]
    ),
)

print("메타데이터 필터가 적용된 검색 결과:")
for i, doc in enumerate(filtered_docs):
    print(f"{i+1}. {doc.page_content}")
    print(f"   메타데이터: {doc.metadata}")
    print("-" * 50)

메타데이터 필터가 적용된 검색 결과:


In [39]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("벡터 데이터베이스란 무엇인가요?")

[Document(metadata={'source': 'blog', 'topic': 'vector_database', 'author': 'tech_blogger', '_id': 'f88deefa-f6da-4842-9284-927ccbedd63a', '_collection_name': 'langchain_collection'}, page_content='벡터 데이터베이스는 임베딩을 저장하고 효율적으로 검색하기 위한 특수 데이터베이스입니다.')]

#### Retriever로 사용

벡터 저장소를 리트리버(Retriever)로 변환하여 LangChain 체인에서 사용할 수 있습니다.



In [40]:
# 벡터 저장소를 리트리버로 변환
retriever = vector_store.as_retriever(
    search_type="similarity",  # 유사도 기반 검색
    search_kwargs={"k": 2}     # 상위 2개 결과 반환
)

# 리트리버로 검색
retrieved_docs = retriever.invoke("LLM과 RAG에 대해 알려주세요")

print("리트리버를 통한 검색 결과:")
for i, doc in enumerate(retrieved_docs):
    print(f"{i+1}. {doc.page_content}")
    print(f"   메타데이터: {doc.metadata}")
    print("-" * 50)

리트리버를 통한 검색 결과:
1. RAG(Retrieval Augmented Generation)는 대규모 언어 모델의 성능을 향상시키기 위해 외부 지식을 가져오는 방법입니다.
   메타데이터: {'source': 'research_paper', 'topic': 'llm', 'author': 'ai_researcher', '_id': 'af4f138f-8166-4920-8c51-eb5c11ea059a', '_collection_name': 'langchain_collection'}
--------------------------------------------------
2. LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션 개발을 위한 프레임워크입니다.
   메타데이터: {'source': 'documentation', 'topic': 'framework', 'author': 'langchain_team', '_id': 'ed20fad1-7741-42ae-917d-66badb4beb17', '_collection_name': 'langchain_collection'}
--------------------------------------------------


### 다양한 검색 모드 구현 (Dense, Sparse, Hybrid)

LangChain과 Qdrant를 함께 사용하면 세 가지 주요 검색 모드를 구현할 수 있습니다:

1. Dense 검색: 임베딩 벡터 기반 검색 (의미적 유사도)
2. Sparse 검색: 키워드 기반 희소 벡터 검색 (키워드 일치)
3. Hybrid 검색: Dense와 Sparse 검색을 조합하여 더 나은 결과 제공

#### Dense 검색 모드

Dense 검색은 기본적인 임베딩 기반 벡터 검색입니다. 이미 위에서 구현한 내용이 Dense 검색에 해당합니다.



In [44]:
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams
from langchain.embeddings import OllamaEmbeddings

# 새로운 컬렉션 생성 (Dense 검색용)
dense_collection = "dense_search_collection"

# 컬렉션 이미 존재하는지 확인 후 생성
client = QdrantClient(":memory:")
if not client.collection_exists(dense_collection):
    client.create_collection(
        collection_name=dense_collection,
        vectors_config=models.VectorParams(
            size=1024,  # OpenAI 임베딩 차원
            distance=Distance.COSINE
        )
    )

# QdrantVectorStore 초기화 (Dense 모드)
dense_store = QdrantVectorStore(
    client=client,
    collection_name=dense_collection,
    embedding=OllamaEmbeddings(model="bge-m3"),
    retrieval_mode=RetrievalMode.DENSE  # 기본값이 Dense이므로 생략 가능
)

# 문서 추가
dense_store.add_documents(documents=documents, ids=ids)

# Dense 검색 수행
dense_results = dense_store.similarity_search(
    "벡터 데이터베이스와 임베딩",
    k=2
)

print("Dense 검색 결과:")
for i, doc in enumerate(dense_results):
    print(f"{i+1}. {doc.page_content}")
    print("-" * 50)

Dense 검색 결과:
1. 벡터 데이터베이스는 임베딩을 저장하고 효율적으로 검색하기 위한 특수 데이터베이스입니다.
--------------------------------------------------
2. Qdrant는 벡터 유사도 검색을 위한 오픈소스 벡터 데이터베이스입니다. 러스트로 작성되었으며 고성능을 자랑합니다.
--------------------------------------------------


#### Sparse 검색 모드

Sparse 검색은 키워드 기반 검색으로, 정확한 단어 일치에 강점이 있습니다. FastEmbed 라이브러리를 사용하여 스파스 임베딩을 생성할 수 있습니다.



In [46]:
%pip install fastembed

Note: you may need to restart the kernel to use updated packages.


In [49]:

# 스파스 임베딩 초기화
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")
    
# 새로운 컬렉션 생성 (Sparse 검색용)
sparse_collection = "sparse_search_collection"
    
# 스파스 벡터 컬렉션 생성
# client.create_collection(
#     collection_name=sparse_collection,
#     vectors_config={},  # Dense 벡터 없음
#     sparse_vectors_config={
#         "sparse": models.SparseVectorParams()
#         }
#     )
    
# QdrantVectorStore 초기화 (Sparse 모드)
sparse_store = QdrantVectorStore(
    client=client,
    collection_name=sparse_collection,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.SPARSE,
    sparse_vector_name="sparse"
    )
    
# 문서 추가
sparse_store.add_documents(documents=documents, ids=ids)
    
# Sparse 검색 수행
sparse_results = sparse_store.similarity_search(
    "HNSW 알고리즘",  # 정확한 키워드로 검색
    k=2
    )
    
print("Sparse 검색 결과:")
for i, doc in enumerate(sparse_results):
    print(f"{i+1}. {doc.page_content}")
    print("-" * 50)

Sparse 검색 결과:
1. HNSW(Hierarchical Navigable Small World)는 고차원 벡터의 근사 최근접 이웃 검색을 위한 알고리즘입니다.
--------------------------------------------------


#### Hybrid 검색 모드

Hybrid 검색은 Dense와 Sparse 검색의 장점을 결합한 방식입니다. 의미적 유사도(Dense)와 키워드 일치(Sparse)를 모두 고려하여 더 나은 검색 결과를 제공합니다.



In [50]:
# 새로운 컬렉션 생성 (Hybrid 검색용)
hybrid_collection = "hybrid_search_collection"
    
# 하이브리드 벡터 컬렉션 생성
client.create_collection(
    collection_name=hybrid_collection,
    vectors_config={
        "dense": models.VectorParams(
            size=1536,  # OpenAI 임베딩 차원
            distance=Distance.COSINE
            )
        },
        sparse_vectors_config={
            "sparse": models.SparseVectorParams()
        }
    )
    
# QdrantVectorStore 초기화 (Hybrid 모드)
hybrid_store = QdrantVectorStore(
    client=client,
    collection_name=hybrid_collection,
    embedding=OpenAIEmbeddings(),
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",
    sparse_vector_name="sparse"
    )
    
# 문서 추가
hybrid_store.add_documents(documents=documents, ids=ids)
    
# Hybrid 검색 수행
hybrid_results = hybrid_store.similarity_search(
    "벡터 데이터베이스 HNSW",  # 의미와 키워드가 혼합된 쿼리
    k=2
    )
    
print("Hybrid 검색 결과:")
for i, doc in enumerate(hybrid_results):
    print(f"{i+1}. {doc.page_content}")
    print("-" * 50)

Hybrid 검색 결과:
1. 벡터 데이터베이스는 임베딩을 저장하고 효율적으로 검색하기 위한 특수 데이터베이스입니다.
--------------------------------------------------
2. HNSW(Hierarchical Navigable Small World)는 고차원 벡터의 근사 최근접 이웃 검색을 위한 알고리즘입니다.
--------------------------------------------------


### 고급 기능 활용 (벡터 양자화, 페이로드 필터링, 그룹화 등)

#### 벡터 양자화

벡터 양자화는 벡터의 저장 공간을 줄이고 검색 성능을 향상시키는 기술입니다. Qdrant는 세 가지 유형의 양자화를 지원합니다:

- 스칼라 양자화: 각 벡터 요소를 개별적으로 압축
- 바이너리 양자화: 각 요소를 1비트로 압축
- 프로덕트 양자화: 벡터를 더 작은 하위 벡터로 분할하여 압축


In [51]:
from qdrant_client.http import models

# 스칼라 양자화를 사용한 컬렉션 생성
scalar_quantization_collection = "scalar_quantization"

client.create_collection(
    collection_name=scalar_quantization_collection,
    vectors_config=models.VectorParams(
        size=512,
        distance=models.Distance.COSINE
    ),
    quantization_config=models.ScalarQuantization(
        scalar=models.ScalarQuantizationConfig(
            type=models.ScalarType.INT8,  # INT8 양자화 (32비트 -> 8비트)
            quantile=0.99,  # 이상치 처리를 위한 분위수
            always_ram=True  # 양자화된 벡터를 항상 RAM에 유지
        )
    )
)

print(f"스칼라 양자화 컬렉션 '{scalar_quantization_collection}' 생성 완료!")

# 프로덕트 양자화를 사용한 컬렉션 생성
product_quantization_collection = "product_quantization"

client.create_collection(
    collection_name=product_quantization_collection,
    vectors_config=models.VectorParams(
        size=512,
        distance=models.Distance.COSINE
    ),
    quantization_config=models.ProductQuantization(
        product=models.ProductQuantizationConfig(
            compression=models.CompressionRatio.X16,  # 16배 압축
            always_ram=True  # 양자화된 벡터를 항상 RAM에 유지
        )
    )
)

print(f"프로덕트 양자화 컬렉션 '{product_quantization_collection}' 생성 완료!")

스칼라 양자화 컬렉션 'scalar_quantization' 생성 완료!
프로덕트 양자화 컬렉션 'product_quantization' 생성 완료!


#### 고급 페이로드 필터링

Qdrant는 다양한 유형의 고급 페이로드 필터링을 지원합니다. 여기서는 복잡한 논리 연산자를 사용한 필터링 예제를 살펴봅니다.



In [57]:
# 테스트용 컬렉션 생성
advanced_filter_collection = "advanced_filter_collection"

# 기존 컬렉션이 있으면 삭제
if client.collection_exists(advanced_filter_collection):
    client.delete_collection(advanced_filter_collection)

client.create_collection(
    collection_name=advanced_filter_collection,
    vectors_config=models.VectorParams(
        size=4,  # 간단한 예제를 위한 작은 차원
        distance=models.Distance.COSINE
    )
)

# 테스트 데이터 생성
test_points = [
    models.PointStruct(
        id=1,
        vector=[0.1, 0.2, 0.3, 0.4],
        payload={
            "name": "제품 A",
            "category": "전자제품",
            "price": 1000,
            "in_stock": True,
            "tags": ["스마트폰", "기술", "새 제품"],
            "rating": 4.5,
            "location": {"lat": 37.5326, "lon": 127.0246}  # 서울 좌표
        }
    ),
    models.PointStruct(
        id=2,
        vector=[0.2, 0.3, 0.4, 0.5],
        payload={
            "name": "제품 B",
            "category": "가구",
            "price": 500,
            "in_stock": True,
            "tags": ["의자", "목재", "할인"],
            "rating": 4.0,
            "location": {"lat": 35.1796, "lon": 129.0756}  # 부산 좌표
        }
    ),
    models.PointStruct(
        id=3,
        vector=[0.3, 0.4, 0.5, 0.6],
        payload={
            "name": "제품 C",
            "category": "전자제품",
            "price": 2000,
            "in_stock": False,
            "tags": ["노트북", "기술", "새 제품"],
            "rating": 4.8,
            "location": {"lat": 37.5326, "lon": 127.0246}  # 서울 좌표
        }
    ),
    models.PointStruct(
        id=4,
        vector=[0.4, 0.5, 0.6, 0.7],
        payload={
            "name": "제품 D",
            "category": "의류",
            "price": 100,
            "in_stock": True,
            "tags": ["티셔츠", "할인", "여름"],
            "rating": 3.5,
            "location": {"lat": 35.8714, "lon": 128.6014}  # 대구 좌표
        }
    )
]

# 테스트 데이터 삽입
client.upsert(
    collection_name=advanced_filter_collection,
    points=test_points
)

print(f"고급 필터링 테스트를 위한 {len(test_points)}개의 포인트가 추가되었습니다.")

# 복합 필터 예제
complex_filter = models.Filter(
    must=[
        # 카테고리가 '전자제품'인 것만 포함
        models.FieldCondition(
            key="category",
            match=models.MatchValue(value="전자제품")
        )
    ],
    should=[
        # 가격이 1500 이상이거나
        models.FieldCondition(
            key="price",
            range=models.Range(gte=1500)
        ),
        # 태그에 '새 제품'이 포함된 것
        models.FieldCondition(
            key="tags",
            match=models.MatchValue(value="새 제품")
        )
    ],
    must_not=[
        # 재고가 없는 것은 제외
        models.FieldCondition(
            key="in_stock",
            match=models.MatchValue(value=False)
        )
    ]
)

# 검색 벡터
query_vector = [0.1, 0.2, 0.3, 0.4]

# 필터를 적용한 검색
filtered_results = client.search(
    collection_name=advanced_filter_collection,
    query_vector=query_vector,
    query_filter=complex_filter,
    with_payload=True,
    limit=10
)

print("\n복합 필터를 적용한 검색 결과:")
for result in filtered_results:
    print(f"ID: {result.id}, 점수: {result.score}")
    print(f"이름: {result.payload.get('name')}")
    print(f"카테고리: {result.payload.get('category')}")
    print(f"가격: {result.payload.get('price')}")
    print(f"재고 여부: {result.payload.get('in_stock')}")
    print(f"태그: {result.payload.get('tags')}")
    print("-" * 40)

고급 필터링 테스트를 위한 4개의 포인트가 추가되었습니다.

복합 필터를 적용한 검색 결과:
ID: 1, 점수: 0.9999999994192575
이름: 제품 A
카테고리: 전자제품
가격: 1000
재고 여부: True
태그: ['스마트폰', '기술', '새 제품']
----------------------------------------
